<img src="./logo_UTN.svg" align="right" width="150" /> 



# Trabajo Semanal Nº 14

## Teoría de los Circuitos 2

#### Juan Camilo Mariscal Gutierrez


## Objetivos:

Diseñe el cuadripolo A para que se comporte como

Filtro pasa bajos Bessel de 3er. orden.

No disipativo.

Normalizado en frecuencia e impedancia.



### Consignas

1 Obtenga la impedancia de entrada al cuadripolo A, cargado con un resistor de 1Ω a la salida.

2 Sintetice A como un cuadripolo escalera.

3 Simule el comportamiento de la red en LTspice graficando S21 y S11 en función de la frecuencia.

4 Explique el comportamiento de A a partir de los valores de S11 en las siguientes frecuencias.
(centro de la banda de paso, frecuencia de corte, transición y centro de la banda de detenida)

5 Modifique el circuito para que la frecuencia de corte sea 2 π 10⁶ rad/s y la resistencia del generador sea 50Ω.

<img src="./cuadripolo.png"  /> 



## Desarrollo numerico 

##### filtro pasa bajos Bessel de orden 3

\begin{equation}
S_{21(s)}= \frac{V_O}{V_I}=\frac{15}{S^3 + 6S^2 +15S + 15} 
\end{equation}

##### sabemos que 

\begin{equation}
|S_{11(s)}|^2 + |S_{21(s)}|^2= 1
\end{equation}


##### Multiplicamos S21 y S21 conjudada para obtener el modulo de S21
\begin{equation}
|S_{21(s)}|^2 = S_{21(s)} * S_{21(-s)}
\end{equation}


\begin{equation}
|S_{21(s)}|^2 =\frac{15}{S^3 + 6S^2 +15S + 15}  * \frac{15}{-S^3 + 6S^2 -15S + 15} 
\end{equation}


\begin{equation}
|S_{21(s)}|^2 =\frac{225}{-S^6 + 6S^4 + 45S + 225}
\end{equation}


\begin{equation}
|S_{11(s)}|^2 = 1 -|S_{21(s)}|^2
\end{equation}

\begin{equation}
|S_{11(s)}|^2 = 1 - \frac{225}{-S^6 + 6S^4 + 45S + 225}
\end{equation}

\begin{equation}
|S_{11(s)}|^2 = \frac{-S^6 + 6S^4 + 45S}{-S^6 + 6S^4 + 45S + 225}
\end{equation}

## Utilizo Python para hallar S11

In [6]:
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
from scipy import signal as signal
from pytc2.sistemas_lineales import bodePlot, pzmap, GroupDelay, analyze_sys, tf2sos_analog, pretty_print_SOS
from pytc2.dibujar import display, dibujar_puerto_entrada, dibujar_funcion_exc_abajo,  dibujar_elemento_serie, dibujar_elemento_derivacion,  dibujar_tanque_derivacion, dibujar_tanque_RC_serie,  dibujar_espacio_derivacion, Capacitor, Resistor, Inductor

## setear en True para ver logs de dev
dev = False

num = np.array([-1, 0, 6, 0, -45, 0, 0])
den = np.array([-1, 0, 6, 0, -45, 0, 225])
roots_num = np.roots(num)
num_poly_result = np.polymul(np.array([-1, roots_num[0]]), np.array([-1, roots_num[1]]))
roots_den = np.roots(den)
sos_den = np.polymul(np.array([-1, roots_den[1]]), np.array([-1, roots_den[2]])).real
fos_den = np.polymul(-1, roots_den[0]).real

if dev == True:
    print(roots_num)
    print(roots_den)
    print(num_poly_result)
    print(num_poly_result.real)
    print(sos_den)
    print(fos_den)


In [5]:
import sympy as sp

from pytc2.remociones import remover_polo_dc, remover_polo_infinito
from pytc2.remociones import remover_valor_en_dc, remover_valor_en_infinito, remover_polo_sigma

from pytc2.sintesis_dipolo import cauer_RC, foster
from pytc2.dibujar import dibujar_cauer_RC_RL, dibujar_foster_derivacion, dibujar_foster_serie
from pytc2.general import print_latex, print_subtitle, a_equal_b_latex_s
from IPython.display import display,  Markdown

s = sp.symbols('s', complex=True)
b0, b1, b2, b3, a0, a1, a2 = sp.symbols('b0 b1 b2 b3 a0 a1 a2', real=True, positive=False)

b0 = num_poly_result[0].real
b1 = num_poly_result[1].real
b2 = num_poly_result[2].real
b3 = 0

a0 = sos_den[0]
a1 = sos_den[1]
a2 = sos_den[2]


s11 = (b0*s**3 + b1*s**2 + b2*s + b3) / ((s + fos_den[0])*(a0*s**2 + a1*s + a2))

display(Markdown(f"""$S_{{11}}={sp.latex(s11)}$"""))


$S_{11}=\frac{1.0 s^{3} + 4.40640532236865 s^{2} + 6.70820393249937 s}{\left(s + 2.32218535462608\right) \left(1.0 s^{2} + 3.67781464537391 s + 6.45943269348336\right)}$

\begin{equation}
S_{11(s)} =\frac{Z_{in} - R_{O1}}{Z_{in} + R_{O1}}  
\end{equation}

##### si RO1 =1  y despejo Zin

\begin{equation}
Z_{in} =\frac{1 + S_{11(s)}}{1 -  S_{11(s)}}  
\end{equation}
 
##### Utilizo valores de S11 con redondeo para hacer los calculos mas faciles 

\begin{equation}
Z_{in} =\frac{1 + \frac{S^3 + 4,406S^2 +6,71S}{S^3 + 6S^2 +15S +15}}{1 - \frac{S^3 + 4,406S^2 +6,71S}{S^3 + 6S^2 +15S +15}}  
\end{equation}

\begin{equation}
Z_{in} =\frac{2S^3 + 10,406s^2 +21,71S +15}{ 1,594S^2 +8,29S +15} 
\end{equation}



## Calculo de los componentes



<img src="./circuito.jpg"  /> 


# Simulacion del circuito en LTSpace

<img src="./simulado.png"  />
<img src="./S21.png"  />
<img src="./S11.png"  />
<img src="./S21 y S11.png"  />

##  Comportamiento de  S11

Podemos observar que en el incio de la banda de paso de la S21 la S11 tiende a -infinito, es decir la red esta adaptada,
la red la potencia reflejada al generador por desaptacion es muy poca.


A medida que nos vamos acercando a la frecuencia de corte la adptacion se va reduciendo.

Si la red empieza a desadaptartese la potencia reflejada al generador aumenta

Una vez que estamos en la banda de detenida S11=1 es decir la red esta totalmente desadaptada en la banda de detenida.
Si toda la potencia incidente se refleja no hay transferencia de potencia hacia la salida,esto perjudica el comportamiento de filtrado.




## Desnormalizacion


\begin{equation}
\omega_o=2\pi10^6
\end{equation}

\begin{equation}
R=50 Ohms
\end{equation}

#### Recalculo de los componentes 

\begin{equation}
L1'=\frac{L1*R}{\omega_o}
\end{equation}
\begin{equation}
L1' = 9,98462629^{-6} H_y
\end{equation}

\begin{equation}
C1'=\frac{C1}{\omega_o * R} 
\end{equation}

\begin{equation}
C1'= 1,759196853^{-9}F
\end{equation}

\begin{equation}
L2'=\frac{L2*R}{\omega_o} 
\end{equation}
\begin{equation}
L2'= 1,530130717^{-6}H_y
\end{equation}

# Circuito desnormalizado

<img src="./simulado_desnor.png"  />
<img src="./S21_desnor.png"  />